Primero importamos las librerías que vamos a necesitar para nuestro modelo de regresión logística

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Ahora introducimos la ruta en la que están ubicados los archivos con la información que se analizará y tambíen aprovechamos para hacer un pequeño ajuste en la visualización del dataframe para meyor versatilidad.

In [13]:
train = pd.read_csv('/home/luis-loreto/Documentos/Proyectos/Titanic/Titanic/Datos/train.csv')
test = pd.read_csv('/home/luis-loreto/Documentos/Proyectos/Titanic/Titanic/Datos/test.csv')

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [54]:
train_c, validacion = train_test_split(train, test_size=0.2, random_state=42)

In [56]:
y = validacion['Survived']

In [57]:
y

709    1
439    0
840    0
720    1
39     1
290    1
300    1
333    0
208    1
136    1
137    0
696    0
485    0
244    0
344    0
853    1
621    1
653    1
886    0
110    0
294    0
447    1
192    1
682    0
538    0
819    0
30     0
673    1
63     0
396    0
877    0
141    1
793    0
235    0
204    1
620    0
23     1
198    1
842    1
739    0
120    0
477    0
837    0
790    0
362    0
86     0
67     0
350    0
210    0
659    0
448    1
820    1
360    0
802    1
280    0
585    1
70     0
299    1
211    1
44     1
408    0
446    1
426    1
168    0
196    0
772    0
72     0
422    0
532    0
591    1
312    0
311    1
889    1
835    1
833    0
605    0
767    0
539    1
357    0
254    0
266    0
66     1
319    1
778    0
174    0
493    0
215    1
309    1
822    0
250    0
165    1
541    0
625    0
388    0
76     0
281    0
712    1
338    1
327    1
321    0
545    0
590    0
291    1
465    0
814    0
442    0
670    1
462    0
286    1
209    1
669    1
2

Iniciamos con el análisis exploratorio. Vemos cuántos registros tiene cada variable. Se puede observar que son 11 variables independientes y una dependiente: Survived. Ésta última será la variable para el aprendizaje en nuestro modelos de machine learning. 

In [21]:
Variables = {
    "Observaciones" : train_c.count(),
    "Missings": train_c.isnull().sum()
}
pd.DataFrame(Variables)

,Observaciones,Missings
PassengerId,712,0
Survived,712,0
Pclass,712,0
Name,712,0
Sex,712,0
Age,572,140
SibSp,712,0
Parch,712,0
Ticket,712,0
Fare,712,0


Vemos que las variables con missings son: Age, Cabin y Embarked. Debemos decidir si vamos a utilizar dichas variables y en su caso qué método de imputación elegir. Por el momento, descartaremos la variable Cabin, pues sus missings representan más de la mitad de las observaciones del dataframe. Usaremos algún método de imputación en las variables Age y Embarked para poder incluirlas en el modelo.

Ahora iniciaremos con la imputación para las variables Age y Embarked. Vamos a crear una función que le asigne valores aleatorios de su determinada variable a las observaciones con valores faltantes-

In [5]:
def hotdeck(data, variable):
    for column in train.columns:
        # Encontrar índices donde hay valores faltantes
        missings = data[data[f"{variable}"].isnull()].index
        
        for x in missings:
            # Seleccionar filas no faltantes
            no_missings = data[f"{variable}"].dropna()
            # Elegir un valor aleatorio de las filas no faltantes
            random_value = no_missings.sample(n=1).values[0]
            # Imputar el valor aleatorio en el índice correspondiente
            data.at[x, f"{variable}"] = random_value
            
    return data

In [22]:
import pandas as pd
import numpy as np

def hotdeck_imputation(data: pd.DataFrame, variable: str) -> pd.DataFrame:
    """
    Realiza una imputación hot-deck aleatoria en una columna específica de un DataFrame.

    Para cada valor faltante en la columna, lo reemplaza con un valor
    seleccionado aleatoriamente de los valores no faltantes en esa misma columna.

    Args:
        data (pd.DataFrame): El DataFrame que contiene los datos.
        variable (str): El nombre de la columna en la que se imputarán los valores.

    Returns:
        pd.DataFrame: El DataFrame con la columna especificada ya imputada.
    """
    # Copia para evitar la advertencia SettingWithCopyWarning
    target_col = data[variable]

    # 1. Encontrar los índices de los valores faltantes
    missing_indices = target_col[target_col.isnull()].index

    # 2. Obtener los valores no faltantes de donde se tomará la muestra
    non_missing_values = target_col.dropna()

    # 3. Generar muestras aleatorias para cada valor faltante
    # Se usa replace=True por si el número de faltantes es mayor que el de existentes
    random_samples = np.random.choice(non_missing_values, size=len(missing_indices), replace=True)

    # 4. Asignar las muestras a los índices faltantes en una sola operación
    data.loc[missing_indices, variable] = random_samples

    return data


In [25]:
train_c = hotdeck(train_c, "Age")
train_c = hotdeck(train_c, "Embarked")

In [26]:
Variables = {
    "Observaciones" : train_c.count(),
    "Missings": train_c.isnull().sum()
}
pd.DataFrame(Variables)

,Observaciones,Missings
PassengerId,712,0
Survived,712,0
Pclass,712,0
Name,712,0
Sex,712,0
Age,712,0
SibSp,712,0
Parch,712,0
Ticket,712,0
Fare,712,0


Ahora vamos a corroborar que nuestro dataframe no tenga valores faltantes.

Ahora iniciaremos con el proceso de One Hot Encoding. Crearemos variables dummies para aquellas con observaciones categóricas, es decir, para Sex, Pclass y Embarked

In [28]:
train_c = pd.get_dummies(train_c, columns = ['Sex', 'Pclass', 'Embarked'], drop_first=True, dtype=float)

Ahora creamos un conjunto de las variables independientes que usaremos y que ya pasaron por un proceso de one hot encoding y otro para la variable dependiente

In [30]:
X_train = train_c[["Age", "SibSp", "Parch", "Fare", "Sex_male", "Pclass_2", "Pclass_3", "Embarked_Q", "Embarked_S"]]
y_train = train_c[["Survived"]]

Independientemente del modelo que vayamos a usar, debemos darle el mismo tratamiento al conjunto de tratamiento. Primero identificaremos si tiene valores faltantes:

In [52]:
validacion['Survived']

KeyError: 'Survived'

In [48]:
validacion = pd.DataFrame(validacion)

In [53]:
validacion

{'Observaciones': Observaciones    12
 Missings         12
 dtype: int64,
 'Missings': Observaciones    0
 Missings         0
 dtype: int64}

In [49]:
validacion = {
    "Observaciones" : validacion.count(),
    "Missings": validacion.isnull().sum()
}
pd.DataFrame(validacion)

,Observaciones,Missings
Observaciones,12,0
Missings,12,0


Vemos que sí, entonces ahora le aplicamos la función que creamos para la imputación. En este caso, se usará para las variables Fare y Age. Cabin se descarta porque los valores faltantes correspondonde a más de la mitad del total.

In [24]:
test = hotdeck(test, "Age")
test = hotdeck(test, "Fare")

In [25]:
v_test = {
    "Observaciones" : test.count(),
    "Missings": test.isnull().sum()
}
pd.DataFrame(v_test)

,Observaciones,Missings
PassengerId,418,0
Pclass,418,0
Name,418,0
Sex,418,0
Age,418,0
SibSp,418,0
Parch,418,0
Ticket,418,0
Fare,418,0
Cabin,91,327


Ahora crearemos nuestro conjunto para la variable dependiente y para la variable independiente, que se encuentra en otro archivo llamado "gender_submission"

Ahora aplicamos el one hot encodging

In [28]:
test = pd.get_dummies(test, columns = ['Sex', 'Pclass', 'Embarked'], drop_first=True, dtype=float)

In [33]:
X_test = test[["Age", "SibSp", "Parch", "Fare", "Sex_male", "Pclass_2", "Pclass_3", "Embarked_Q", "Embarked_S"]]
y_test = pd.read_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/gender_submission.csv')

In [30]:
y_test = pd.read_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/gender_submission.csv')

Ahora vamos a guardar los archivos ya preparados en nuestra carpeta "Datos"

In [34]:
X_train.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/X_train.csv', index=False)
y_train.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/y_train.csv', index=False)
X_test.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/X_test.csv', index=False)
y_test.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/y_test.csv', index=False)